In [43]:
state = {"Global Difficulty": 1000,
         "Aggregate Hashpower": 2500, # Per Second
         "Regions": 3,
         "Zones per Region": 3,
         "Target Time": 2}

params = {"Block Difficulty Multiples": {"Prime": 1,
                                         "Region": .5,
                                         "Zone": .25},
        "Block Hash Multiplier Lognormal Distribution": {"mu": 0, "std": .05}
        }

In [48]:
# Add a log normal distribution for pertubing the hash needed
import numpy as np

def create_block_hashes(state, params):

    # Get the baseline block difficulties without randomness
    prime_block_hashes = [state["Global Difficulty"] * params["Block Difficulty Multiples"]["Prime"]]
    region_block_hashes = [state["Global Difficulty"] * params["Block Difficulty Multiples"]["Region"]] * state["Regions"]
    zone_block_hashes = [state["Global Difficulty"] * params["Block Difficulty Multiples"]["Zone"]]* state["Regions"] * state["Zones per Region"]

    # Merge and order
    block_hashes = []
    while len(region_block_hashes) > 0:
        for _ in range(state["Zones per Region"]):
            block_hashes.append(zone_block_hashes.pop())
        block_hashes.append(region_block_hashes.pop())
    block_hashes.append(prime_block_hashes.pop())

    # Add in randomness
    block_hashes = np.array(block_hashes)
    mutlipliers = np.random.lognormal(0, .05, len(block_hashes))
    block_hashes = block_hashes * mutlipliers
    block_hashes = block_hashes.round().astype(int)

    # Get cumulative sum for later computation of how far the aggregate hash rate gets us
    block_hashes_cs = np.cumsum(block_hashes)

    return block_hashes, block_hashes_cs

def compute_progress(state, params, block_hashes, block_hashes_cs):
    total_hashpower = state["Aggregate Hashpower"] * state["Target Time"]
    if  total_hashpower >= block_hashes_cs[-1]:
        print("Prime block is mined!")
        print("Time to mine = {}".format(block_hashes_cs[-1] / total_hashpower* state["Target Time"]))
        mined_blocks = block_hashes
        block_hashes, block_hashes_cs = [], []
    else:
        print("Prime block not mined!")
    
    print("The following difficulties are emitted from mined blocks: {}".format(mined_blocks))
    return block_hashes, block_hashes_cs

In [49]:
block_hashes = []
for _ in range(100):
    if len(block_hashes) == 0:
        block_hashes, block_hashes_cs = create_block_hashes(state, params)
    block_hashes, block_hashes_cs = compute_progress(state, params, block_hashes, block_hashes_cs)

Prime block is mined!
Time to mine = 1.9016
The following difficulties are emitted from mined blocks: [ 273  218  238  546  245  238  247  482  252  266  261  478 1010]
Prime block is mined!
Time to mine = 1.8688
The following difficulties are emitted from mined blocks: [245 240 245 499 236 249 235 499 262 249 262 473 978]
Prime block is mined!
Time to mine = 1.9108
The following difficulties are emitted from mined blocks: [ 237  240  229  516  249  240  247  516  247  228  238  528 1062]
Prime block is mined!
Time to mine = 1.8956
The following difficulties are emitted from mined blocks: [ 258  251  268  487  272  247  245  468  255  255  259  462 1012]
Prime block is mined!
Time to mine = 1.882
The following difficulties are emitted from mined blocks: [255 247 271 547 227 247 249 476 259 236 240 502 949]
Prime block is mined!
Time to mine = 1.864
The following difficulties are emitted from mined blocks: [257 246 247 493 260 241 239 499 244 258 272 477 927]
Prime block is mined!
Time 

In [4]:
# Bisection to find the blocks processed

In [5]:
# If leftover blocks, the values get scaled by the change in difficulty